In [18]:
import torch
import os, sqlite3
from flask import Flask, session, request,  g, jsonify, render_template, flash, redirect, url_for
from werkzeug.utils import secure_filename
from functools import wraps
from passlib.hash import sha256_crypt as sha
from wtforms import Form, BooleanField, StringField, PasswordField, validators,TextAreaField,IntegerField

from transformers import ViTImageProcessor, ViTModel, AutoImageProcessor, AutoModelForImageClassification,  AutoModelForCausalLM, AutoTokenizer
from PIL import Image
import requests
import torch
from datasets import load_dataset



In [56]:


Database = 'MainDatabase.db'
UPLOAD_FOLDER = os.path.join('static' ,'uploads')
ALLOWED_EXTENSIONS = set(['jpg', 'jpeg'])


In [57]:
UPLOAD_FOLDER

'static\\uploads'

In [3]:
device = "cuda" # the device to load the model onto

model_nlp = AutoModelForCausalLM.from_pretrained("stabilityai/stable-code-instruct-3b")
tokenizer = AutoTokenizer.from_pretrained("stabilityai/stable-code-instruct-3b")

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.02s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:

# processor = ViTImageProcessor.from_pretrained('nateraw/food')
image_processor = AutoImageProcessor.from_pretrained('nateraw/food')
# model = ViTModel.from_pretrained('nateraw/food')
model_vision = AutoModelForImageClassification.from_pretrained('nateraw/food')

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [5]:
def get_top_k_pred_labels(im, k=3):

    inputs = image_processor(im, return_tensors="pt")
    with torch.no_grad():
        logits = model_vision(**inputs).logits
    # Get indices of the top two predicted labels
    top_k_values, top_k_indices = logits.topk(k, dim=-1)

    # Extract the indices as Python scalars
    predicted_labels = top_k_indices.tolist()

    predicted_labels_with_labels = [[model_vision.config.id2label[label] for label in labels] for labels in predicted_labels]

    return predicted_labels_with_labels[0]

In [6]:
# TESTING VIT 
filename = "Test/pizza_slice.jpg"
with Image.open(filename) as image:
    image.load()
predicted_labels_with_labels = get_top_k_pred_labels(image)
predicted_labels_with_labels

['pizza', 'bruschetta', 'garlic_bread']

In [119]:
def get_nlp_op(messages):
    prompt = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
    inputs = tokenizer([prompt], return_tensors="pt").to(model_nlp.device)

    tokens = model_nlp.generate(
        **inputs,
        max_new_tokens=1024,
        temperature=0.5,
        top_p=0.95,
        top_k=100,
        do_sample=True,
        use_cache=True
    )

    output = tokenizer.batch_decode(tokens[:, inputs.input_ids.shape[-1]:], skip_special_tokens=False)[0]

    constant_string = '<|im_end|>\n<|endoftext|>'

    # Check if the original string ends with the constant string
    if output.endswith(constant_string):
        # Remove the constant string from the end of the original string
        modified_string = output[:-len(constant_string)]
        # print(modified_string)
   
    return modified_string

In [8]:
message_template = [
       {
        "role": "system",
        "content": "You are a helpful and polite assistant for diabetes patients",
    }] 

In [66]:
def generate_food_nlp_prompt(items):
    food_message_template = [
        {
            "role": "system",
            "content": "You are a helpful and polite assistant for diabetes patients. They want to know if the food they are eating is good for diabetes patient",
        },
        {
            "role": "user",
            "content": f"is eating {' and '.join(items[:-1])} and {items[-1]} good for me"
        }
        ] 
    return food_message_template

In [120]:
# TEST
test_prompt = generate_food_nlp_prompt(['sugar', 'dessert', 'pancake'])
print(test_prompt)
res = get_nlp_op(test_prompt)
res

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'role': 'system', 'content': 'You are a helpful and polite assistant for diabetes patients. They want to know if the food they are eating is good for diabetes patient'}, {'role': 'user', 'content': 'is eating sugar and dessert and pancake good for me'}]


'It is important to remember that diabetes is a chronic condition, and it is important to eat a balanced diet that is healthy for your overall health. It is generally recommended that adults eat no more than 2,000 calories per day, with a majority of that coming from fruits, vegetables, lean proteins, and whole grains. Sugar is a key component of many processed foods, so it’s important to be aware of how much sugar is in the food you’re eating. Foods high in sugar include baked goods, candy, and many types of fruit.  If you are concerned about your blood sugar levels, it is important to monitor your diet and eat a healthy, balanced diet.  For more information, you can visit the US Department of Agriculture’s Food Guide to Diets and Nutrition.'

In [151]:
app = Flask(__name__, static_url_path="", static_folder="static")
app.secret_key = os.urandom(12)

def allowed_file(filename):     
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER


def login_required(f):
    @wraps(f)
    def decorated_function(*args, **kwargs):
        if session.get("username") is None:
            flash(" Please login ",'danger')

            return redirect(url_for("login",user=session.get("username"), next=request.url))
        return f(*args, **kwargs)

    return decorated_function
def admin_required(f):
    @wraps(f)
    def decorated_function(*args, **kwargs):
        if (session.get("usertype") !="admin"):
            return redirect(url_for("index",user=session.get("username"), next=request.url))
        return f(*args, **kwargs)

    return decorated_function


def get_db():
    db = getattr(g, '_database', None)
    if db is None:
        db = g._database = sqlite3.connect(Database)
    return db


def query_db(query, args=(), one=False):  # used to retrieve values from the table
    cur = get_db().execute(query, args)
    rv = cur.fetchall()
    cur.close()
    return (rv[0] if rv else None) if one else rv


def execute_db(query, args=()):  # executes a sql command like alter table and insert
    conn = get_db()
    cur = conn.cursor()
    cur.execute(query, args)
    conn.commit()
    cur.close()

# class ReservationForm(Form):
#     firstname=StringField('First Name',[validators.Length(min=2,max=25),validators.DataRequired()])
#     lastname=StringField('First Name',[validators.Length(min=2,max=25)])
#     email=StringField('Email',[validators.Email,validators.DataRequired()])
#     time=IntegerField('Time',[validators.DataRequired()])
#     seats=IntegerField('Number of seats',[validators.DataRequired()])
#     address=StringField('Address',[validators.DataRequired()])
#     message=TextAreaField('Message for the chef',[validators.optional(),validators.Length(max=200)])
#     accept_tos=BooleanField('I accept TOS',[validators.DataRequired()])


@app.route('/')
def index():
    if(session.get("usertype")!="admin"):
        return render_template('index.html',user=session.get("username"),)
    if(session.get("usertype")=="admin"):
        return render_template('admin_index.html',user=session.get("username"))
@app.route('/about')
def about():
    return render_template('about.html',user=session.get("username"))

@app.route('/menu')
def menu():
    data = query_db("select name,category,description,option1,option2,option3,price1,price2,price3 from menu")
    return render_template('services.html',user=session.get("username"),alldata=data)

@app.route('/services')
def services():
    data = ""
    return render_template('services.html',user=session.get("username"),alldata=data)


@app.route('/team')
def team():
    return render_template('team.html',user=session.get("username"))

@app.route('/reservation',methods=['POST','GET'])
def reservation():

    if request.method=="GET":
        return render_template('reservation.html',user=session.get("username"))
    else:
        submission ={}
        submission["firstname"]=request.form["firstname"]
        submission["lastname"]=request.form["lastname"]
        submission["email"]=request.form["email"]
        submission["time"]=request.form["time"]
        submission["address"]=request.form["address"]
        submission["city"]=request.form["city"]
        submission['notes']=request.form['notes']
        submission['seats']=request.form['seats']

        execute_db("insert into reservations values(?,?,?,?,?,?,?,?)", (
        submission["firstname"],
        submission["lastname"],
        submission["email"],
        submission["time"],
        submission["address"],
        submission["city"],
        submission['notes'],
        submission['seats']
        ))
    flash(" Reservation created ","success")
    return render_template('reservation.html',user=session.get("username"))


@app.route('/contact')
def contact():
    return render_template('contact.html',user=session.get("username"))

@app.route('/elements')
def elements():
    return render_template('elements.html')



@app.route('/login',methods=['POST','GET'])
def login():
    if request.method == "GET":
        return render_template("login.html",user=session.get("username"))
    else:
        error = None
        username = request.form["username"]
        password = request.form["password"]

        passdatabase = query_db("select password from users where username = ?", (username,))
        if passdatabase == []:
            flash(" User does not exist ", "danger")
            return render_template("login.html",user=session.get("username"))
        usertype=query_db("select usertype from users where username = ?", (username,))

        if  (sha.verify(password,passdatabase[0][0])):
            flash(" Login successful ","success")
            session["username"] = username
            session["usertype"]=usertype[0][0]
            return redirect(url_for('index'))
        else:
            flash( " Incorrect Password ", "danger")
            return render_template("login.html",user=session.get("username"))
        
@app.route('/admin_register',methods=['POST','GET'])
def admin_register():
    if request.method=="GET":
        return render_template('admin_register.html',user=session.get("username"))
    else:
        submission = {}
        submission["email"] = request.form["email"]

        submission["password"] = request.form["password"]
        submission["conf_pass"] = request.form["conf_pass"]
        usertype="admin"
        if submission["password"] != submission["conf_pass"]:
            flash("Passwords don't match", "danger")
            return render_template("admin_register.html",user=session.get("username"))

        if query_db("select username from users where username = ?", (submission["email"],)) != []:
            flash("Email already taken", "danger")
            return render_template("admin_register.html",user=session.get("username"))

        password = sha.encrypt(submission["password"])
        uid=100000000*np.random.random()
        uid=int(uid)

        execute_db("insert into users values(?,?,?,?)", (
            uid,
            submission["email"],

            password,
            usertype
        ))

        flash(" User Created ", "success")
    return redirect(url_for("login"))

@app.route('/register',methods=['POST','GET'])
def register():
    if request.method=="GET":
        return render_template('register.html',user=session.get("username"))
    else:
        submission = {}
        submission["email"] = request.form["email"]
        submission["firstname"]=request.form["firstname"]
        submission["lastname"] = request.form["lastname"]
        submission["address"] = request.form["address"]
        submission["contact"]=request.form["contact"]
        submission["password"] = request.form["password"]
        submission["conf_pass"] = request.form["conf_pass"]
        usertype="customer"
        if submission["password"] != submission["conf_pass"]:
            flash("Passwords don't match", "danger")
            return render_template("register.html",user=session.get("username"))

        if query_db("select username from users where username = ?", (submission["email"],)) != []:
            flash("Email already taken", "danger")
            return render_template("register.html",user=session.get("username"))

        password = sha.encrypt(submission["password"])
        uid=100000000*np.random.random()
        uid=int(uid)

        execute_db("insert into users values(?,?,?,?)", (
            uid,
            submission["email"],

            password,
            usertype
        ))
        execute_db("insert into customers values(?,?,?,?,?,?)",(
            uid,
            submission["firstname"],
            submission["lastname"],
            submission["email"],
            submission["contact"],
            submission["address"]
        ))
        flash(" User Created ", "success")
    return redirect(url_for("login"))

@app.route('/logout')
def logout():
    session.clear()
    flash(" Logout successful ", "success")
    return redirect(url_for('login'))


@app.route('/employee')
def employee():
    return render_template('team.html',user=session.get("username"))

@app.route('/layout')
def layout():
    return render_template('layout.html')

@app.route("/profile")
@login_required
def profile():
    
    return render_template('dashboard.html')




@app.route("/dashboard")
def dashboard():
      # response from the server
    return render_template('dashboard.html')

@app.route("/upload_file", methods = ["POST"])
def upload_file():
    
    return url_for("food_detector")

@app.route("/food_detector", methods = ["GET", "POST"])
@login_required
def food_detector():
    if request.method == 'POST':
        file = request.files['img']
        filename = secure_filename(file.filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        img = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        print("img = " + str(img))


        with Image.open(img) as image:
            image.load()
        predicted_labels_with_labels = get_top_k_pred_labels(image)

        nlp_prompt = generate_food_nlp_prompt(predicted_labels_with_labels)
        formatted_string = ', '.join(item.replace('_', ' ') for item in predicted_labels_with_labels[:-1]) + ' and ' + predicted_labels_with_labels[-1].replace('_', ' ')

        generated_text = get_nlp_op(nlp_prompt)
        img = 'uploads/' + filename
        print("img sent = " + str(img))
        return render_template('food_detector.html', img=img, foods = formatted_string, text = generated_text)
    
    return render_template('food_detector.html')

chats = [
    # {'sender': 'You', 'message': 'Hello!'},
    {'sender': 'Alex', 'message': 'Hi there! How can I assist you?'}
    ]

@app.route('/chatbot', methods=['POST', 'GET'])
@login_required
def chatbot():
    

    if request.method == 'POST':
        
        user_input = request.form["message"]
        
        # Generate response
        user_msg = message_template + [{"role": "user", "content":user_input}]
        response = get_nlp_op(user_msg)
        print(response)
        chats.append({'sender': 'You', 'message': str(user_input)})
        chats.append({'sender': 'Alex', 'message': str(response)})
        
        return render_template('chatbot.html', chats=chats)

    
    return render_template('chatbot.html', chats=chats)
    

    

In [152]:
if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Apr/2024 22:26:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2024 22:26:10] "GET /css/linearicons.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:26:10] "GET /css/font-awesome.min.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:26:10] "GET /css/bootstrap.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:26:10] "GET /css/magnific-popup.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:26:10] "GET /css/nice-select.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:26:10] "GET /css/animate.min.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:26:10] "GET /css/jquery-ui.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:26:10] "GET /css/owl.carousel.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:26:10] "GET /css/main.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:26:10] "GET /img/logo.png HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:26:10] "GET /img/about-img.png HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/202

Diabetes is a chronic condition that affects how the body processes food and does not allow the body to use glucose for energy., causing the body to become tired and have cravings for sugary foods. It is a disease that can be managed with a healthy lifestyle and proper treatment.


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
127.0.0.1 - - [03/Apr/2024 22:28:41] "POST /chatbot HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2024 22:28:41] "GET /css/linearicons.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:28:41] "GET /css/font-awesome.min.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:28:41] "GET /css/bootstrap.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:28:41] "GET /css/magnific-popup.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:28:41] "GET /css/nice-select.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:28:41] "GET /css/animate.min.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:28:41] "GET /css/jquery-ui.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:28:41] "GET /css/owl.carousel.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:28:41] "GET /css/main.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:28:41] "GET /img/logo.png HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:28:41] "GET /js/vendor/jquery-2.2.4.min.js HTTP/1.1" 

The best way to keep your blood sugar levels in check is to make sure you are eating a balanced diet and consuming plenty of fruits, vegetables, whole grains, lean proteins, and healthy fats.

Additionally, it is important to monitor your blood sugar levels regularly. You can do this by tracking your blood sugar levels with a handheld glucose meter, or by using a self-monitoring tool such as a glucose analyzer or a mobile app. Regularly checking your blood sugar levels will help you identify and address any potential issues early on.

You should also be upfront with your healthcare provider about your blood sugar levels, and make sure they are aware of any medications or supplements you are taking. This will help them tailor a personalized treatment plan to your individual needs.

Finally, it is important to stay active in managing your blood sugar levels. This can include things like maintaining a healthy weight, exercising regularly, and making healthy lifestyle choices such as getti

127.0.0.1 - - [03/Apr/2024 22:28:53] "GET /food_detector HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2024 22:28:53] "GET /css/linearicons.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:28:53] "GET /css/bootstrap.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:28:53] "GET /css/font-awesome.min.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:28:53] "GET /css/magnific-popup.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:28:53] "GET /css/nice-select.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:28:53] "GET /css/animate.min.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:28:53] "GET /css/jquery-ui.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:28:53] "GET /css/owl.carousel.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:28:53] "GET /css/main.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:28:53] "GET /img/logo.png HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:28:53] "GET /js/vendor/jquery-2.2.4.min.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:28:53] "GET /js/vendor/boot

img = static\uploads\burger.jpg


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
127.0.0.1 - - [03/Apr/2024 22:29:44] "POST /food_detector HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2024 22:29:44] "GET /css/linearicons.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:29:44] "GET /css/font-awesome.min.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:29:44] "GET /css/magnific-popup.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:29:44] "GET /css/bootstrap.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:29:44] "GET /css/nice-select.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:29:44] "GET /css/animate.min.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:29:44] "GET /css/jquery-ui.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:29:44] "GET /css/owl.carousel.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:29:44] "GET /css/main.css HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:29:44] "GET /img/logo.png HTTP/1.1" 304 -
127.0.0.1 - - [03/Apr/2024 22:29:44] "GET /uploads/burger.jpg HTTP/1.1" 200 -

img sent = uploads/burger.jpg
